# Codebook

In [39]:
import os, glob, re
import pandas as pd

In [36]:
def cleanSummaries(summary):
    summary = re.sub(r'([HL]L:)', r'\n\1', summary)    
    summary = re.sub(r'([HL]L:\s+)\n', r'\1', summary)
    summary = re.sub(r'Number:\s\d', '', summary)    
    summary = re.sub(r'Subject: Highlight', '', summary)
    summary = re.sub(r'Author: steph', '', summary)
    summary = re.sub(r'Date:\s\d\d-\w+-\d\d\s\d\d?:\d\d:\d\d\s[AP]M', '', summary)
    return summary

In [42]:
ptrn = os.path.join('.', 'data', '**', '*.txt')
data = pd.DataFrame(columns=['org', 'article', 'analysis', 'level', 'code'])
rep = lambda s, n: [ s for i in range(n) ]
for fn in glob.iglob(ptrn, recursive=True):
    org, article, analysis = fn.split('/')[2:5]
    with open(fn, encoding='latin-1') as f:
        txt = cleanSummaries(f.read())
    codes = re.findall(r'[HL]L:[^\n]+', txt)
    df = pd.DataFrame({
        'org': rep(org, len(codes)),
        'article': rep(article, len(codes)),
        'analysis': rep(analysis, len(codes)),
        'level': [ c[0:2] for c in codes ],
        'code': [ c[4:] for c in codes ]
    })
    data = data.append(df)

data.head()

,org,article,analysis,level,code
0,buzzfeednews,2019-04-democratic-candidate-codonors,Summary of Comments on analyze-campaign-codono...,LL,Import CSV data
1,buzzfeednews,2019-04-democratic-candidate-codonors,Summary of Comments on analyze-campaign-codono...,LL,Import ASCII data
2,buzzfeednews,2019-04-democratic-candidate-codonors,Summary of Comments on analyze-campaign-codono...,HL,Convert ambiguous data to tabular format
3,buzzfeednews,2019-04-democratic-candidate-codonors,Summary of Comments on analyze-campaign-codono...,LL,Peak at data
4,buzzfeednews,2019-04-democratic-candidate-codonors,Summary of Comments on analyze-campaign-codono...,LL,"Join datasets, right"


In [55]:
data.groupby(['level', 'code']).count()['org'] \
    .to_frame('count') \
    .sort_values(['level', 'count'], ascending=False) \
    .reset_index()

,level,code,count
0,LL,Peak at data,8
1,LL,Peak at data,7
2,LL,"Peak at data, 5",7
3,LL,Count total records,6
4,LL,Count total records,4
5,LL,Create constant column,4
6,LL,"Export data, CSV",4
7,LL,"Join datasets, union",4
8,LL,"Sort dataset, 1, DESC",4
9,LL,"Join dataset, left",3
